In [29]:
import numpy as np
import pandas as pd

In [30]:
Kyoto_label = pd.read_csv("0_Labels_Kyoto.csv")
WDC_label = pd.read_csv("0_Labels_WDC.csv")
Kyoto_temperature = pd.read_csv("1_Temperature_Kyoto.csv")
WDC_temperature = pd.read_csv("1_Temperature_WDC.csv")

In [31]:
TEMPERATURE_PERIOD = 10
PREDICTION_PERIOD = 10
total_records = np.zeros((0, TEMPERATURE_PERIOD + 1))

In [32]:
for year in range(1949, 2017): # WDC Tempreature Data
    curr_flowering_DOY = WDC_label.iloc[year-1949, 1]
    curr_records = np.zeros((curr_flowering_DOY - TEMPERATURE_PERIOD, TEMPERATURE_PERIOD + 1))
    
    if len(WDC_temperature[WDC_temperature["Year"] == year]) > 0:
        start_row = WDC_temperature[WDC_temperature["Year"] == year].index[0]

        for day in range(TEMPERATURE_PERIOD, curr_flowering_DOY):
            curr_records[day-TEMPERATURE_PERIOD, :-1] = WDC_temperature.iloc[day-TEMPERATURE_PERIOD+start_row:day+start_row, 1].to_numpy()
            curr_records[day-TEMPERATURE_PERIOD, -1] = curr_flowering_DOY - day if (curr_flowering_DOY - day <= PREDICTION_PERIOD) else 0
        
        total_records = np.vstack((total_records, curr_records))

for year in range(1881, 2021): # Kyoto Tempreature Data
    curr_flowering_DOY = Kyoto_label.iloc[year-1881, 1]
    if curr_flowering_DOY != 0:
        curr_records = np.zeros((curr_flowering_DOY - TEMPERATURE_PERIOD, TEMPERATURE_PERIOD + 1))
        start_row = Kyoto_temperature[Kyoto_temperature["Year"] == year].index[0]

        for day in range(TEMPERATURE_PERIOD, curr_flowering_DOY):
            curr_records[day-TEMPERATURE_PERIOD, :-1] = Kyoto_temperature.iloc[day-TEMPERATURE_PERIOD+start_row:day+start_row, 1].to_numpy()
            curr_records[day-TEMPERATURE_PERIOD, -1] = curr_flowering_DOY - day if curr_flowering_DOY - day <= PREDICTION_PERIOD else 0
        total_records = np.vstack((total_records, curr_records))

In [33]:
np.savetxt("processed_data.csv", total_records, delimiter=",", fmt=['%.2f']*10+['%d'])